In [1]:
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import CORe50
from avalanche.models import SimpleCNN
from avalanche.training import GDumb
from avalanche.logging import InteractiveLogger
from avalanche.logging.text_logging import TextLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    ram_usage_metrics,
    timing_metrics,
)

c:\Users\USER\anaconda3\envs\avalanche-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Eval_metics
eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    gpu_usage_metrics(gpu_id=0,experience=True),
    ram_usage_metrics(experience=True),
    timing_metrics(experience=True),
    loggers=[InteractiveLogger(),
                TextLogger(open('gdumb_results.txt', 'w'))]
)

# model
model = SimpleCNN(num_classes=50)

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# CL Benchmark Creation
core50 = CORe50(scenario="nc", mini=True) # original 128x128, mini 32x32
# scenario : nbatches
# "ni": 8, "nc": 9, "nic": 79, "nicv2_79": 79, "nicv2_196": 196, "nicv2_391": 391
train_stream = core50.train_stream
test_stream = core50.test_stream

E:\python\CL\avalanche_root\avalanche\training\plugins\evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified


In [3]:
# Continual learning strategy
cl_strategy = GDumb(
    model,
    optimizer,
    criterion,
    train_mb_size=128, train_epochs=10,
    eval_mb_size=128, device=device,
    evaluator=eval_plugin
)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))

-- >> Start of training phase << --
100%|██████████| 2/2 [00:02<00:00,  1.11s/it]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0100
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0139
100%|██████████| 2/2 [00:00<00:00, 14.69it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0050
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0139
100%|██████████| 2/2 [00:00<00:00, 14.27it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0000
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████| 2/2 [00:00<00:00, 14.80it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0000
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0000
100%|██████████| 2/2 [00:00<00:00, 14.80it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.0200
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.0278
100%|██████████| 2/2 [00:00<00:00, 14.38it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train